In [57]:
import numpy as np
import copy
import pickle
import math
from pennylane.labs import resource_estimation as re
from utils import vibronic_fragments_modebased, count_nonzero_Q_terms, get_norm_value
from pennylane.labs.trotter_error import vibronic_fragments
from pennylane.labs.trotter_error import RealspaceMatrix

# Resource Estimation for Quantum Algorithm for Non-Adiabatic Dynamics

### Cost Metric

We take the Toffoli gate count, alongside qubit count, as our primary cost metric because the Toffoli, being a non Clifford gate, is a resource-intensive component in many quantum error correction schemes. Clifford gates can be implemented cheaply in many fault-tolerant architectures.

Moreover, Toffoli‐based constructions are used for the arithmetic and modular‐arithmetic subroutines (e.g., adders, squaring, multiplier circuits) in quantum chemistry and optimization routines, thus, minimizing Toffoli count directly reduces the distillation workload and overall circuit depth. 

In [58]:
my_gs = {'X', 'Z', 'Y', 'S', 'Hadamard', 'CNOT','T', 'Toffoli'}

def print_Qubit_Toff(resources):
    '''Function for printing the resources for a circuit'''
    qubit_count = resources.qubit_manager.total_qubits
    toffoli_count = resources.clean_gate_counts.get("Toffoli", 0)
    
    if toffoli_count > 9999:
        toffoli_count = f"{toffoli_count:.3E}"
    
    print(f'Qubit Count = {qubit_count}')
    print(f'Toffoli Count = {toffoli_count}')

### Circuits used in Algorithm

Phase gradient circuit used for the resource state:

In [59]:
b = 20 # Number of Qubit used for the resource state

phase_grad_wires = [f"pg_{i}" for i in range(b)]
def initial_circ(b, phase_grad_wires):
    re.ResourcePhaseGradient(num_wires=b, wires=phase_grad_wires)

# We change the default single qubit rotation precision to 1e-15 so that the error
# from the roations can be neglected for error in the phase added.
epsilon = 1e-15

res_initial = re.estimate_resources(
    initial_circ, 
    gate_set=my_gs, 
    single_qubit_rotation_error=epsilon,)(b, phase_grad_wires)


Circuit for implementing linear ($Q_r$) terms:

In [60]:
def Q_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    for i in range(k):
        ctrl_wire = [mode_wires[i]]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires+ctrl_wire)

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))
     
    re.ResourceIdentity(wires = scratch_wires + total_mode_wires[k:])

Circuit for implementing quadratic ($Q_r^2$) terms:

In [61]:
def Qsquared_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires)

    for i in range(2*k):
        ctrl_wire = [f"s_{i}"]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

    re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires))

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))

    re.ResourceIdentity(wires = total_mode_wires[k:])

Circuit for implementing Bilinear ($Q_r Q_s$) terms:

In [62]:
def QrQs_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, mode2_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    re.ResourceOutMultiplier(a_num_qubits=k, b_num_qubits=k, wires=scratch_wires+mode_wires+mode2_wires)

    for i in range(2*k):
        ctrl_wire = [f"s_{i}"]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

    re.ResourceAdjoint(re.ResourceOutMultiplier(a_num_qubits=k, b_num_qubits=k, wires=scratch_wires+mode_wires+mode2_wires))

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))

    re.ResourceIdentity(wires = total_mode_wires[2*k:])

Changing the default resource decomposition for QFT:

In [63]:
'''
We use a different implementation of the QFT algorithm which is more resource efficient than
the default textbook implemention used in Pennylane Resource Estimation,
This comes from <https://arxiv.org/abs/1803.04933v2>.
'''

def AQFT_resource_decomp(num_wires, **kwargs):
    ceil_log_n = math.ceil(math.log2(num_wires))
    aux_qubit_count = num_wires + 3*ceil_log_n - 4  # (Section IV. )
    
    toff = re.ResourceT.resource_rep()
    toff_count = 8 * num_wires*(ceil_log_n - 1)

    gate_list = [
        re.AllocWires(aux_qubit_count),
        re.GateCount(toff, toff_count),
        re.FreeWires(aux_qubit_count),
    ]
    return gate_list

re.ResourceQFT.set_resources(AQFT_resource_decomp)
# re.ResourceQFT.set_resources(re.ResourceQFT.default_resource_decomp)  # reset to default QFT cost

Circuit for applying a Unitary rotation on the electronic subspace:

In [64]:
re.ResourceMultiplexer.set_resources(re.ResourceMultiplexer.phase_grad_resource_decomp)

def digonalizing_circuit(N, electronic_wires):
    re.ResourceQubitUnitary(num_wires=N, precision=1e-10, wires=electronic_wires)

In [65]:
def kinetic_cir(b, M, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    for i in range(M):
        re.ResourceQFT(num_wires=k, wires=total_mode_wires[i*k: k*(i + 1)])

    for i in range(M):
        mode_wires = total_mode_wires[i*k: k*(i + 1)]
        re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires)

        for j in range(2*k):
            ctrl_wire = [f"s_{j}"]
            target_wires = coeff_wires[:b-j] + phase_grad_wires[:b-j]
            re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - j), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

        re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires))

    for i in range(M):
        re.ResourceAdjoint(re.ResourceQFT(num_wires=k), wires=total_mode_wires[i*k: k*(i + 1)])

    re.ResourceIdentity(wires=electronic_wires)

## Resource Estimation for Systems of Interest

Loading the Hamiltonian:

In [66]:
''' 
Uncomment the molecule that resource estimation should be performed for.
'''

# mol = 'no4a_monomer' 
# mol = 'no4a_dimer'            
# mol = 'anthra-c60_ct'
# mol = 'anthracene_6s_66m'
# mol = 'pentacene_16s_102m'
mol = 'maleimide_5s_24m_bilin'
# mol = 'maleimide_5s_24m_nobilin'

k = 4 # Number of Qubits for discretization of each mode
filehandler = open(f'model_params/{mol}.pkl', 'rb')
omegas, couplings = pickle.load(filehandler)
filehandler.close()

QVC = bool(2 in couplings)
# Check if the model has contain Quadratic Couplings
omegas = np.array(omegas)
lambdas = couplings[0]
alphas = couplings[1]

N, M = alphas.shape[0], alphas.shape[2]
log_N = math.ceil(math.log2(N))

if QVC:
    betas = couplings[2]
else:
    betas = np.zeros((N, N, M, M))

print(f'Loaded {mol} Hamiltonian with {M} modes and {N} states.')

Loaded maleimide_5s_24m_bilin Hamiltonian with 24 modes and 5 states.


/var/folders/zv/cfr06cz541b53yzvds885cb00000gq/T/ipykernel_31805/2517105548.py:15: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  omegas, couplings = pickle.load(filehandler)


Naming the wires used in the circuits:

In [67]:
electronic_wires = [f"e_{i}" for i in range(log_N)]
coeff_wires = [f"c_{i}" for i in range(b)]
total_mode_wires = [f"m_{i}" for i in range(k*M)]
mode_wires = total_mode_wires[:k]
scratch_wires = [f"s_{i}" for i in range(2*k)] 

Estimating the cost of implementing linear ($Q_r$) terms:

In [68]:
res_Q = re.estimate_resources(Q_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)
print_Qubit_Toff(res_Q)

Qubit Count = 166
Toffoli Count = 146


Estimate the cost of implementing quadratic ($Q_r^2$) terms:

In [69]:
res_Qsquared = re.estimate_resources(Qsquared_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)
print_Qubit_Toff(res_Qsquared)

Qubit Count = 166
Toffoli Count = 272


Estimate the cost of implementing bilinear ($Q_r Q_s$) terms:

In [70]:
mode2_wires = total_mode_wires[k:2*k]
res_QrQs = re.estimate_resources(QrQs_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, mode2_wires, scratch_wires)
print_Qubit_Toff(res_QrQs)

Qubit Count = 166
Toffoli Count = 310


### Fragmention of Hamiltonian (OG)

In [71]:
fragments_OG = vibronic_fragments(N, M, omegas, [lambdas, alphas, betas])
print(f'Decomposed Hamiltonian into {len(fragments_OG)} fragments using the Orginal Grouping(OG) method')

Decomposed Hamiltonian into 9 fragments using the Orginal Grouping(OG) method


Cost of implementing the kinetic fragment:

In [72]:
res_kinetic = re.estimate_resources(kinetic_cir, gate_set=my_gs)(b, M, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)

Qubit_count = res_kinetic.qubit_manager.total_qubits
print_Qubit_Toff(res_kinetic)

Qubit Count = 166
Toffoli Count = 6384


The `count_nonzero_Q_terms` function counts the number of $Q_r$, $Q_r^2$ and $Q_r Q_s$ terms in a given fragment. These counts determine how many times the circuits `Q_cir`, `Qsquared_cir` and `QrQs_cir`will be required for implementing each fragment of the potential part.


In [73]:
res_list_OG = [res_kinetic]

for i in range(len(fragments_OG)-1):
    num_Qr, num_Qr2, num_QrQs = count_nonzero_Q_terms(fragments_OG[i])
    res_frag = num_Qr * res_Q + num_Qr2 * res_Qsquared + num_QrQs * res_QrQs
    res_list_OG.append(res_frag)

### Single Trotter Step Cost (OG)

The two most expensive fragments are implemented once and the othes are implemented twice for second order trotter:

In [74]:
res_expensive_frags = sorted(res_list_OG, key=lambda res_frag: res_frag.clean_gate_counts.get("Toffoli", 0),reverse=True)[:2]
total_OG_res = res_expensive_frags[0] + res_expensive_frags[1] 

for res in res_list_OG:
    if res not in res_expensive_frags:
        total_OG_res += res * 2

print_Qubit_Toff(total_OG_res)

Qubit Count = 166
Toffoli Count = 3.119E+04


### Fragmentation of Hamiltonian (Mode Based)

In [75]:
fragments_mode = vibronic_fragments_modebased(N, M, omegas, [lambdas, alphas, betas], 'FC')

num_frags_mode = len(fragments_mode)
print(f'Decomposed Hamiltonian into {num_frags_mode} fragments using the mode based fragmentation method')

Decomposed Hamiltonian into 22 fragments using the mode based fragmentation method


In [76]:
res_list_mode = [res_kinetic]

for i in range(len(fragments_mode)-1):
    num_Qr, num_Qr2, num_QrQs = count_nonzero_Q_terms(fragments_mode[i])
    res_frag = num_Qr * res_Q + num_Qr2 * res_Qsquared + num_QrQs * res_QrQs
    res_list_mode.append(res_frag)

res_expensive_frags = sorted(res_list_mode, key=lambda res_frag: res_frag.clean_gate_counts.get("Toffoli", 0),reverse=True)[:2]
total_mode_res = res_expensive_frags[0] + res_expensive_frags[1] 

for res in res_list_mode:
    if res not in res_expensive_frags:
        total_mode_res += res * 2

Next, we add the cost of the Unitaries acting on the electronic register:

In [77]:
res_unitary = re.estimate_resources(digonalizing_circuit,gate_set=my_gs,)(N, electronic_wires)

total_mode_res += res_unitary
print_Qubit_Toff(total_mode_res)

Qubit Count = 166
Toffoli Count = 2.557E+04


### Number of Trotter Steps

Loading spectral norm of the trotter error operator, $||\varepsilon||$:

In [78]:
CSV_FILENAME = "norms.csv"
norm = get_norm_value(mol=f'{mol}.pkl', K=2**k, M=M, filepath=CSV_FILENAME)

Norm not found!


Then the required number of trotter steps for a simulation for time t and an error tolerance of $\epsilon$ is:

$$
n = t^{1.5}\cdot \sqrt{ \frac{||\varepsilon||}{\epsilon}}.
$$

In [79]:
def num_steps(norm, req_error, total_time):
    return math.ceil(total_time**1.5 * (norm / req_error)**0.5)

total_time = 152 # in ev, time that we want to get the cost for (100 fs).
req_error = 0.01 # 1 percent error

if norm is not None:
    nsteps = num_steps(norm, req_error, total_time)
    print(f'{nsteps} Trotter steps are required for a {100*req_error}% error tolerance for simulation of the hamiltonian.')